In [20]:
import sys
sys.path.append("/Users/mistryk2/Packages/nudobe/src/")
sys.path.append("/Users/mistryk2/Packages/nudobe/")

import numpy as np
import nudobe
from nudobe import EFT, functions, constants
from constants import G_F
from functions import *
from EFT import LEFT
import pandas as pd

In [21]:
def get_WCs_leptoquark(alpha_SL  = 0,
            alpha_VL  = 0,
            alpha_SR  = 0,
            alpha_VR  = 0,
            epsilon_S = 0,
            epsilon_V = 0, 
            M_V       = 1*TeV, 
            M_S       = 1*TeV
           ):
    WC_LQ = {}
    
    #see arXiv: hep-ph/9603213 for the definition of eps_I, alpha_I
    WC_LQ["SL(6)"] =  1/(G_F*np.sqrt(2)) * epsilon_V/(M_V**2)
    WC_LQ["SR(6)"] =  1/(G_F*np.sqrt(2)) * epsilon_S/(M_S**2)
    WC_LQ["VL(6)"] = +1/(G_F*np.sqrt(2)) * np.sqrt(2) * (alpha_SL/(M_S**2) + alpha_VL/(M_V**2))
    WC_LQ["VR(6)"] = -1/(G_F*np.sqrt(2)) * (alpha_SR/(M_S**2) + alpha_VR/(M_V**2))

    return(WC_LQ)

In [22]:
KamLAND_limit = 3.6e26
NME="SM"

limits = {}    
limits["alpha_SL"]  = np.sqrt(LEFT(get_WCs_leptoquark(alpha_SL  = 1), method = NME).t_half("136Xe")/KamLAND_limit)
limits["alpha_VL"]  = np.sqrt(LEFT(get_WCs_leptoquark(alpha_VL  = 1), method = NME).t_half("136Xe")/KamLAND_limit)
limits["alpha_SR"]  = np.sqrt(LEFT(get_WCs_leptoquark(alpha_SR  = 1), method = NME).t_half("136Xe")/KamLAND_limit)
limits["alpha_VR"]  = np.sqrt(LEFT(get_WCs_leptoquark(alpha_VR  = 1), method = NME).t_half("136Xe")/KamLAND_limit)
limits["epsilon_S"] = np.sqrt(LEFT(get_WCs_leptoquark(epsilon_S = 1), method = NME).t_half("136Xe")/KamLAND_limit)
limits["epsilon_V"] = np.sqrt(LEFT(get_WCs_leptoquark(epsilon_V = 1), method = NME).t_half("136Xe")/KamLAND_limit)

display(limits)

{'alpha_SL': 9.745206538656799e-09,
 'alpha_VL': 9.745206538656799e-09,
 'alpha_SR': 1.774691217457988e-06,
 'alpha_VR': 1.774691217457988e-06,
 'epsilon_S': 3.2101019790992533e-09,
 'epsilon_V': 3.2101019790992533e-09}

In [23]:
def get_masses(m_min           = 0, 
                ordering        = "NO", 
                Majorana_phases = [0,0], 
                v_R             = 10*TeV, 
                v_L             = 0.1*eV, 
                m_heavy         = np.array([10,12,13])*TeV,
                theta_L         = 0,
               ):
    U = U_PMNS(alpha=Majorana_phases)
    m = m_min
    
    if ordering == "NO" or ordering == "NH":
        m1 = m
        m2 = np.sqrt(m1**2+m21)
        m3 = np.sqrt(m2**2+m32)
    elif ordering == "IO" or ordering == "IH":
        m3 = m
        m2 = np.sqrt(m3**2-m32IO)
        m1 = np.sqrt(m2**2-m21)

    
    #diagonal light neutrino masses
    m_nu = np.diag([m1,m2,m3])*1e-9
    
    #diagonal heavy neutrino masses
    m_nu_R = np.diag(m_heavy)
    
    #non-diagonal light neutrino mass
    M_nu = U@m_nu@U.T
    
    #non-diagonal heavy neutrino mass
    M_nu_R = U@m_nu_R@U.T
    
    #inverse matrices
    M_nu_R_inv = np.linalg.inv(M_nu_R)
    
    M_nu_R_dagger_inv = np.linalg.inv(M_nu_R.T.conjugate())
    
    #non-diagonal yukawa matrices
    M_R = M_nu_R.T.conjugate()/(np.sqrt(2)*v_R)
    M_L = M_R.T.conjugate()
    
    M_nu_L = (np.sqrt(2) * v_L * np.exp(1j*theta_L) * M_L)
    
    #get the dirac yukawa matrix eq.57
    M_nu_D = U@m_nu_R@scipy.linalg.sqrtm(v_L/v_R*np.exp(1j*theta_L) * np.diag([1,1,1])
                                                          - np.linalg.inv(m_nu_R)@m_nu+0j)@U.T
    #return results
    return(m_nu, m_nu_R, M_nu_R, M_nu_R_inv, M_nu_R_dagger_inv, M_nu_D, M_nu_L)


# Gets the Wilson Coeficients at SMEFT
def GetWCs_mLRSM(m_min=0, v_R=10*TeV, v_L=0.1*eV, m_heavy=np.array([10,12,13])*TeV, theta_L=0, xi=0):
      m_nu, m_nu_R, M_nu_R, M_nu_R_inv, M_nu_R_dagger_inv, M_nu_D, M_nu_L = get_masses(
                                                                                    m_min           = 0, 
                                                                                    ordering        = "NO", 
                                                                                    Majorana_phases = [0,0], 
                                                                                    v_R             = v_R, 
                                                                                    v_L             = v_L, 
                                                                                    m_heavy         = m_heavy,
                                                                                    theta_L         = theta_L,
                                                                                    )


      # SMEFT
      C5         = 1/vev**2 * (M_nu_D.T@M_nu_R_inv@M_nu_D-M_nu_L)[0,0]
      C_LeudPhi7 = np.sqrt(2)/vev * (1/v_R**2 *(V_ud_R.conjugate()) * M_nu_D.T@M_nu_R_inv)[0,0]
      C_LPhiDe7  = 2j * xi*np.exp(1j * alpha) / (1 + xi**2) * C_LeudPhi7 / V_ud_R.conjugate()
      C_eeud9    = 0.88 * (-1 / (2 * v_R**4) * V_ud_R**2 * (M_nu_R_dagger_inv + 2 / m_DR**2 * M_nu_R))[0,0]
      C_eePhiud9 = -4 * (xi * np.exp(-1j * alpha)) / (1 + xi**2) * C_eeud9/V_ud_R
      C_eePhiD9  = 4 * xi**2 * np.exp(-2j * alpha) / ((1 + xi**2)**2) * C_eeud9/V_ud_R**2

      return {"LH(5)"      : C5, 
            "LeudH(7)"   : C_LeudPhi7, 
            "LHDe(7)"    : C_LPhiDe7, 
            "ddueue(9)"  : 4*np.conj(C_eeud9), 
            "deueH2D(9)" : -2 * np.conj(C_eePhiud9), 
            "eeH4D2(9)"  : -np.conj(C_eePhiD9)
            }

# Matches SMEFT wilson coeficients down to m_W scale using Table in Appendix A and Matching scheme in Appendix B
def GetWCsLEFT_mLRSM(m_min=0, v_R=10*TeV, v_L=0.1*eV, m_heavy=np.array([10,12,13])*TeV, theta_L=0, xi=0):
      SMEFT_WCs = GetWCs_mLRSM(0, v_R, v_L, m_heavy, theta_L, xi)

      m_bb   = -vev**2 * SMEFT_WCs["LH(5)"]
      C_VR6  = (vev**3/np.sqrt(2)) * SMEFT_WCs["LeudH(7)"].conjugate()
      C_VL6  = - (vev**3 * 1j) /np.sqrt(2) *V_ud * SMEFT_WCs["LHDe(7)"].conjugate()
      C_1R9p = - (vev**5 / 4) *  SMEFT_WCs["ddueue(9)"].conjugate()
      C_4R9  = -vev**5 * (V_ud/2) * SMEFT_WCs["deueH2D(9)"].conjugate()
      C_1R9  = -vev**5 * (V_ud/2) * SMEFT_WCs["eeH4D2(9)"].conjugate()

      return {
            "m_bb"       : m_bb,
            "VR(6)"      : C_VR6,
            "VL(6)"      : C_VL6,
            "1R(9)prime" : C_1R9p,
            "4R(9)"      : C_4R9,
            "1R(9)"      : C_1R9
      }

In [24]:
# ------- mLRSM ---------
#masses
v_R  = 10*TeV
v_L  = 0.1*eV
m_DR = 4*TeV

#heavy neutrino masses
m_heavy = np.array([10,12,13])*TeV

#mixings
V_ud_R  = V_ud
V_ud_L  = V_ud
xi      = m_b/m_t
theta_L = 0
alpha   = 0

In [35]:
limits = {}    
limits["m_min"]  = np.sqrt(LEFT(GetWCsLEFT_mLRSM(m_min=0, v_R=10*TeV, v_L=0.1*eV, m_heavy=np.array([10,12,13])*TeV, theta_L=0, xi=0), method = NME).t_half("136Xe")/KamLAND_limit)
limits["v_R"]    = np.sqrt(LEFT(GetWCsLEFT_mLRSM(m_min=0, v_R=10*TeV, v_L=0.1*eV, m_heavy=np.array([10,12,13])*TeV, theta_L=0, xi=0), method = NME).t_half("136Xe")/KamLAND_limit)

display(limits)

{'m_min': 10.904192862837492, 'v_R': 10.90240808634651}